<a href="https://colab.research.google.com/github/Deepika-Ch19/Auto-encoders/blob/main/network_intrusion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the libraries

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Read the csv file

In [20]:
import pandas as pd
filepath='/content/drive/MyDrive/Midterm_53_group.csv'
df = pd.read_csv(filepath)
df.head()

,Time,Source,No.,Destination,Protocol,Length,Info
0,0.000000,192.167.8.166,1,192.167.255.255,NBNS,92,Name query NB WPAD<00>
1,0.784682,192.167.8.166,2,192.167.255.255,NBNS,92,Name query NB WPAD<00>
2,1.169060,VMware_8a:5c:e6,3,Broadcast,ARP,60,Who has 192.167.7.175? Tell 192.167.0.1
3,2.167949,VMware_8a:5c:e6,4,Broadcast,ARP,60,Who has 192.167.7.175? Tell 192.167.0.1
4,3.170095,VMware_8a:5c:e6,5,Broadcast,ARP,60,Who has 192.167.7.175? Tell 192.167.0.1


In [4]:
print(df.columns.tolist())


['Time', 'Source', 'No.', 'Destination', 'Protocol', 'Length', 'Info']


In [21]:
df = pd.get_dummies(df, columns=['Protocol'], drop_first=True)
df['is_query'] = df['Info'].str.contains('query', case=False).astype(int)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['Time', 'Length']] = scaler.fit_transform(df[['Time', 'Length']])
df = df.drop(columns=['Info'])
df.head()


,Time,Source,No.,Destination,Length,Protocol_BROWSER,Protocol_DHCP,Protocol_DNS,Protocol_HTTP,Protocol_ICMP,...,Protocol_NBNS,Protocol_OCSP,Protocol_RARP,Protocol_SSLv2,Protocol_STUN,Protocol_TCP,Protocol_TLSv1,Protocol_TLSv1.2,Protocol_TLSv1.3,is_query
0,0.000000,192.167.8.166,1,192.167.255.255,0.001006,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,1
1,0.000624,192.167.8.166,2,192.167.255.255,0.001006,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,1
2,0.000930,VMware_8a:5c:e6,3,Broadcast,0.000362,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
3,0.001725,VMware_8a:5c:e6,4,Broadcast,0.000362,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
4,0.002522,VMware_8a:5c:e6,5,Broadcast,0.000362,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0


In [22]:
from sklearn.ensemble import IsolationForest
X = df[['Time', 'Length']]
model = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
df['anomaly'] = model.fit_predict(X)
print(df['anomaly'].value_counts())
df[df['anomaly'] == -1].head()

anomaly
 1    374460
-1     19676
Name: count, dtype: int64


,Time,Source,No.,Destination,Length,Protocol_BROWSER,Protocol_DHCP,Protocol_DNS,Protocol_HTTP,Protocol_ICMP,...,Protocol_OCSP,Protocol_RARP,Protocol_SSLv2,Protocol_STUN,Protocol_TCP,Protocol_TLSv1,Protocol_TLSv1.2,Protocol_TLSv1.3,is_query,anomaly
0,0.000000,192.167.8.166,1,192.167.255.255,0.001006,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,-1
1,0.000624,192.167.8.166,2,192.167.255.255,0.001006,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,-1
2,0.000930,VMware_8a:5c:e6,3,Broadcast,0.000362,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,-1
3,0.001725,VMware_8a:5c:e6,4,Broadcast,0.000362,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,-1
4,0.002522,VMware_8a:5c:e6,5,Broadcast,0.000362,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,-1


In [23]:
from sklearn.svm import OneClassSVM
features = df[['Time','Length']]
small_sample = features.sample(n=500, random_state=42)
svm_small = OneClassSVM(kernel='rbf', nu=0.05, gamma='scale')
df_sample = df.copy()
df_sample.loc[small_sample.index, 'svm_anomaly'] = svm_small.fit_predict(small_sample)
print(df['anomaly'].value_counts())
df[df['anomaly']==-1].head()

anomaly
 1    374460
-1     19676
Name: count, dtype: int64


,Time,Source,No.,Destination,Length,Protocol_BROWSER,Protocol_DHCP,Protocol_DNS,Protocol_HTTP,Protocol_ICMP,...,Protocol_OCSP,Protocol_RARP,Protocol_SSLv2,Protocol_STUN,Protocol_TCP,Protocol_TLSv1,Protocol_TLSv1.2,Protocol_TLSv1.3,is_query,anomaly
0,0.000000,192.167.8.166,1,192.167.255.255,0.001006,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,-1
1,0.000624,192.167.8.166,2,192.167.255.255,0.001006,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,-1
2,0.000930,VMware_8a:5c:e6,3,Broadcast,0.000362,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,-1
3,0.001725,VMware_8a:5c:e6,4,Broadcast,0.000362,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,-1
4,0.002522,VMware_8a:5c:e6,5,Broadcast,0.000362,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,-1


In [24]:
from sklearn.neighbors import LocalOutlierFactor
features = df[['Time', 'Length']]
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
df_sample = df.copy()
df_sample['lof_anomaly'] = lof.fit_predict(features)
print(df_sample['lof_anomaly'].value_counts())
df_sample[df_sample['lof_anomaly'] == -1].head()


lof_anomaly
 1    374429
-1     19707
Name: count, dtype: int64


,Time,Source,No.,Destination,Length,Protocol_BROWSER,Protocol_DHCP,Protocol_DNS,Protocol_HTTP,Protocol_ICMP,...,Protocol_RARP,Protocol_SSLv2,Protocol_STUN,Protocol_TCP,Protocol_TLSv1,Protocol_TLSv1.2,Protocol_TLSv1.3,is_query,anomaly,lof_anomaly
45,0.030948,0.0.0.0,46,255.255.255.255,0.006478,False,True,False,False,False,...,False,False,False,False,False,False,False,0,-1,-1
54,0.032089,VMware_8a:5c:e6,55,Broadcast,0.000362,False,False,False,False,False,...,False,False,False,False,False,False,False,0,-1,-1
56,0.032394,192.167.8.65,57,192.167.255.255,0.001006,False,False,False,False,False,...,False,False,False,False,False,False,False,1,-1,-1
57,0.032686,VMware_8a:a0:c6,58,Broadcast,0.000362,False,False,False,False,False,...,False,False,False,False,False,False,False,0,-1,-1
58,0.032848,192.167.7.175,59,192.167.255.255,0.001368,False,False,False,False,False,...,False,False,False,False,False,False,False,0,-1,-1
